In [1]:
import pyodbc


In [2]:
# Conectar ao SQL Server
conn = pyodbc.connect(
    "DRIVER={SQL Server};"
    "SERVER=localhost\SQLEXPRESS;"
    "DATABASE=Cabeleireiro2;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()


In [3]:
# Coletar dados do usuário
nome = input('Nome (opcional): ').strip() or None
idade = int(input('Idade: '))
bairro = input('Bairro: ').strip()
dia_da_semana = input('Dia da semana que corta: ').strip()
corte = input('Nome do corte: ').strip()
cabeleireiro = input('Nome do cabeleireiro: ').strip()
faz_barba = input('Faz a barba? (S/N): ').strip().upper() == 'S'
faz_sobrancelha = input('Faz sobrancelha? (S/N): ').strip().upper() == 'S'
quantas_vezes_mes = int(input('Quantas vezes corta no mês? '))
sugestao = input('Algo a acrescentar? ').strip() or None



Nome (opcional): luiz
Idade: 26
Bairro: vila alpina
Dia da semana que corta: quarta
Nome do corte: 
Nome do cabeleireiro: lucas
Faz a barba? (S/N): n
Faz sobrancelha? (S/N): s
Quantas vezes corta no mês? 2
Algo a acrescentar? n


In [4]:
# Inserir o corte na tabela de Cortes (se ainda não existir)
cursor.execute("IF NOT EXISTS (SELECT 1 FROM Cortes WHERE NomeCorte = ?) INSERT INTO Cortes (NomeCorte) VALUES (?)", (corte, corte))
conn.commit()


In [5]:
# Obter o ID do corte inserido
cursor.execute("SELECT CorteID FROM Cortes WHERE NomeCorte = ?", (corte,))
corte_id = cursor.fetchone()[0]


In [6]:
# Inserir o cabeleireiro na tabela de Cabeleireiros (se ainda não existir)
cursor.execute("IF NOT EXISTS (SELECT 1 FROM Cabeleireiros WHERE NomeCabeleireiro = ?) INSERT INTO Cabeleireiros (NomeCabeleireiro) VALUES (?)", (cabeleireiro, cabeleireiro))
conn.commit()


In [7]:
# Obter o ID do cabeleireiro inserido
cursor.execute("SELECT CabeleireiroID FROM Cabeleireiros WHERE NomeCabeleireiro = ?", (cabeleireiro,))
cabeleireiro_id = cursor.fetchone()[0]



In [8]:
# Inserir o cliente na tabela de Clientes
cursor.execute("""
    INSERT INTO Clientes (Nome, Idade, Bairro, DiaDaSemana, QuantasVezesMes, Sugestao)
    VALUES (?, ?, ?, ?, ?, ?)
""", (nome, idade, bairro, dia_da_semana, quantas_vezes_mes, sugestao))
conn.commit()



In [9]:
# Obter o ID do cliente inserido
cursor.execute("SELECT ClienteID FROM Clientes WHERE Nome = ? AND Bairro = ?", (nome, bairro))
cliente_id = cursor.fetchone()[0]



In [10]:
# Inserir o agendamento na tabela de Agendamentos
cursor.execute("""
    INSERT INTO Agendamentos (ClienteID, CorteID, CabeleireiroID, FazBarba, FazSobrancelha)
    VALUES (?, ?, ?, ?, ?)
""", (cliente_id, corte_id, cabeleireiro_id, faz_barba, faz_sobrancelha))
conn.commit()
print("Dados inseridos com sucesso!")

# Fechar a conexão
cursor.close()
conn.close()

Dados inseridos com sucesso!


In [2]:
import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

# Substitua os valores de luizh, password, host e database com os seus próprios.
app.config['SQLALCHEMY_DATABASE_URI'] = 'mssql+pyodbc://DIDICO\\luizh:@localhost\\SQLEXPRESS/Cabeleireiro2?driver=ODBC+Driver+17+for+SQL+Server'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False  # Desabilita o aviso de modificações

db = SQLAlchemy(app)

# Modelo (tabela)
class Cabeleireiro(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    nome = db.Column(db.String(80), nullable=False)
    idade = db.Column(db.Integer, nullable=False)

    def __repr__(self):
        return f'<Cabeleireiro {self.nome}>'

@app.route('/')
def index():
    # Criando as tabelas (se ainda não existirem)
    db.create_all()

    # Inserir dados de exemplo
    if not Cabeleireiro.query.first():
        novo_cabeleireiro = Cabeleireiro(nome='João', idade=25)
        db.session.add(novo_cabeleireiro)
        db.session.commit()

    # Exibir todos os cabeleireiros cadastrados
    cabeleireiros = Cabeleireiro.query.all()
    return '<br>'.join([f'{c.nome}, {c.idade} anos' for c in cabeleireiros])

if __name__ == '__main__':
    app.run(debug=True)

ModuleNotFoundError: No module named 'Flask'